# Region Selection

In this tutorial, we will demonstrate how to:

- Set up a Dask client and load an object catalog
- Select data from regions in the sky
    - cone
    - radec box
    - polygon
    - constructed MOC (multi-order coverage map)

## Introduction

Large astronomical surveys contain a massive volume of data. Billion-object, multi-terabyte-sized catalogs are challenging to store and manipulate because they demand state-of-the-art hardware. Processing them is expensive, both in terms of runtime and memory consumption, and doing so on a single machine has become impractical. LSDB is a solution that enables scalable algorithm execution. It handles loading, querying, filtering, and crossmatching astronomical data (of HATS format) in a distributed environment. 

In [ ]:
import lsdb

## 1. Load a catalog

We create a basic dask client, and load an existing HATS catalog - the ZTF DR22 catalog.

In [ ]:
from dask.distributed import Client

client = Client(n_workers=4, memory_limit="auto")
client

In [ ]:
ztf_object_path = "https://data.lsdb.io/hats/ztf_dr22/ztf_lc"
ztf_object = lsdb.open_catalog(ztf_object_path)
ztf_object

## 2. Selecting a region of the sky

There are 3 common types of spatial filters to select a portion of the sky: cone, polygon and box.

Filtering consists of two main steps:

- A __coarse__ stage, in which we find what pixels cover our desired region in the sky. These may overlap with the region and only be partially contained within the region boundaries. This means that some data points inside that pixel may fall outside of the region.

- A __fine__ stage, where we filter the data points from each pixel to make sure they fall within the specified region.

The `fine` parameter allows us to specify whether or not we desire to run the fine stage, for each search. It brings some overhead, so if your intention is to get a rough estimate of the data points for a region, you may disable it. It is always executed by default.

```
catalog.box_search(..., fine=False)
catalog.cone_search(..., fine=False)
catalog.polygon_search(..., fine=False)
```

Throughout this notebook, we will use the Catalog's `plot_pixels` method to display the HEALPix of each resulting catalog as filters are applied.

In [ ]:
ztf_object.plot_pixels(plot_title="ZTF_DR14 - pixel map")

## 3. Cone search

A cone search is defined by center `(ra, dec)`, in degrees, and radius `r`, in arcseconds.

In [ ]:
ztf_object_cone = ztf_object.cone_search(ra=-60.3, dec=20.5, radius_arcsec=5 * 3600)
ztf_object_cone

In [ ]:
ztf_object_cone.plot_pixels(plot_title="ZTF_DR14 - cone pixel map")

## 4. The Search object

To perform a search on a catalog, there are two modes: a shape-specific call, or passing a search object to the `search()` method. The above case uses the cone shape call. 

Using a search object can be useful if you intend to re-use the shape for filtering multiple catalogs. We also provide some basic plotting for cone and box searches. The 5 degree cone search is outlined in red in the below plot.


In [ ]:
from lsdb import ConeSearch

cone_search = ConeSearch(ra=-60.3, dec=20.5, radius_arcsec=5 * 3600)

In [ ]:
ztf_object.plot_pixels(plot_title="ZTF_DR14 - pixel map")
cone_search.plot(fc="#00000000", ec="red")

## 5. Polygon search

A polygon search is defined by convex polygon with vertices `[(ra1, dec1), (ra2, dec2)...]`, in degrees.

In [ ]:
vertices = [(-60.5, 15.1), (-62.5, 18.5), (-65.2, 15.3), (-64.2, 12.1)]
ztf_object_polygon = ztf_object.polygon_search(vertices)
ztf_object_polygon

In [ ]:
ztf_object_polygon.plot_pixels(plot_title="ZTF_DR14 - polygon pixel map")

## 6. Box search

A box search can be defined by right ascension and declination bands `[(ra1, ra2), (dec1, dec2)]`.

In [ ]:
ztf_object_box = ztf_object.box_search(ra=[-65, -60], dec=[12, 15])
ztf_object_box

In [ ]:
ztf_object_box.plot_pixels(plot_title="ZTF_DR14 - box pixel map")

## 7. Complex and MOC filters

We can stack a several number of filters, which are applied in sequence. For example, `catalog.box_search().polygon_search()` should result in a perfectly valid HATS catalog containing the objects that match **both** filters.

However, we can also get the MOC (or Multi-Order coverage map) of the regions, and perform a filter based on that region. Check out our [notebook on constructing complex regions in a MOC in the HATS documentation](https://hats.readthedocs.io/en/latest/notebooks/moc_usage.html). Here, we will use similar regions as that notebook. 

However, we set the `max_depth` to the same granularity as the highest HEALPix order in our catalog. This ensures that we're not getting data partitions that are completely outside of the regions, just due to low resolution MOCs.

In [ ]:
from hats.pixel_math import region_to_moc
from hats.inspection.visualize_catalog import plot_moc

max_depth = ztf_object.hc_structure.get_max_coverage_order()

box_moc = region_to_moc.box_to_moc(ra=[10, 45], dec=[-30, -5], max_depth=max_depth)
cone_moc = region_to_moc.cone_to_moc(ra=-60.3, dec=20.5, radius_arcsec=15 * 3600, max_depth=max_depth)
polygon_moc = region_to_moc.polygon_to_moc([(5, -10), (50, -10), (30, 18)], max_depth=max_depth)
union_moc = polygon_moc.union(cone_moc, box_moc)
plot_moc(union_moc)

In [ ]:
ztf_object_moc = ztf_object.moc_search(union_moc)
ztf_object_moc

In [ ]:
ztf_object_moc.plot_pixels()

## Closing the Dask client

In [ ]:
client.close()

## About

**Authors**: Sandro Campos and Melissa DeLucchi

**Last updated on**: August 29, 2025

If you use `lsdb` for published research, please cite following [instructions](https://docs.lsdb.io/en/stable/citation.html).